In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


In [4]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.5 MB/s eta 0:00:00


In [5]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.1/932.1 kB 29.3 MB/s eta 0:00:00


# Requirements

In [6]:
import torch

from torch.optim import Adam
from torch_geometric.nn import ComplEx, DistMult, RotatE, TransE
from torch_geometric.data import Data

from base import OptimizerConfig, cross_validation
from base import SimplePytorchData, SimplePytorchDataTrainTestSplit
from base import SimpleTrainer, SimpleTester
from src.config import SimpleClassifierConfig, GraphAutoEncoderConfig, KGEConfig
from src.features import get_relations, get_entities, get_associations, get_homogeneous_graph, get_kge_pair_embedd_for_training_data
from src.models import SimpleMDAClassifier, SimpleMDAClassifierFactory
from src.utils import train_test_sampler, prj_logger
from torch_geometric.nn import GCNConv

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

# ComplEx

## Config

In [9]:
kge_optimizer_config = OptimizerConfig()
kge_optimizer_config.optimizer = torch.optim.Adam
kge_optimizer_config.lr = 0.01
kge_optimizer_config.batch_size = 1000
kge_optimizer_config.n_epoch = 30
kge_optimizer_config.exp_name = "Optimizer for Graph Auto Encoder"
kge_optimizer_config.device = device

In [10]:
kge_model_config = KGEConfig()
kge_model_config.kge = ComplEx
kge_model_config.hidden_channels = 32

## Embedding

In [11]:
md_embed = get_kge_pair_embedd_for_training_data(kge_model_config, kge_optimizer_config)

2024-02-11 16:45:51,550 [INFO] Calling get_node2vec_pair_embedd on cpu device ...
2024-02-11 16:45:51,552 [INFO] Calling get_homogeneous_graph
2024-02-11 16:45:54,582 [INFO] homogeneous data : Data(x=[66911, 1], edge_index=[2, 633662])
2024-02-11 16:45:54,591 [INFO] Calling get_kge_embedd on cpu device ...
2024-02-11 16:45:54,593 [INFO] Calling get_knowledge_graph
2024-02-11 16:45:55,686 [INFO] knowledge graph data : Data(num_nodes=66911, edge_index=[2, 633662], edge_type=[633662])
2024-02-11 16:45:55,707 [INFO] Setting num relations and num nodes for kge config to 39 and 66911
2024-02-11 16:45:55,710 [INFO] Creating KGE model ...
2024-02-11 16:45:55,712 [INFO] Initialing MDATransE with model_config {'model_name': None}
2024-02-11 16:45:55,785 [INFO] Training KGE ...
2024-02-11 16:45:55,786 [INFO] Running KGETrainer with Optimizer for Graph Auto Encoder
2024-02-11 16:45:55,789 [INFO] Creating <class 'torch.optim.adam.Adam'> with lr : 0.01
2024-02-11 16:45:55,792 [INFO] moving model to 

# Classification

## Data

In [12]:
associations = get_associations()
y = torch.tensor(associations['increased'].tolist(), dtype=torch.float32).reshape(-1, 1).to(device)

In [13]:
# Train Test Split
train_indices, test_indices = train_test_sampler(y.shape[0], 0.7)

data = SimplePytorchData(md_embed, y)
train_data = SimplePytorchData(md_embed[train_indices], y[train_indices])
test_data = SimplePytorchData(md_embed[test_indices], y[test_indices])

2024-02-11 17:02:15,267 [INFO] Initializing SimplePytorchData with X shape : torch.Size([898, 64]) and y shape : torch.Size([898, 1])
2024-02-11 17:02:15,271 [INFO] Initializing SimplePytorchData with X shape : torch.Size([628, 64]) and y shape : torch.Size([628, 1])
2024-02-11 17:02:15,273 [INFO] Initializing SimplePytorchData with X shape : torch.Size([270, 64]) and y shape : torch.Size([270, 1])


## Classifier

In [14]:
simple_classifier_config = SimpleClassifierConfig()
simple_classifier_config.model_name = "simple classifier"
simple_classifier_config.input_dim = md_embed.shape[1]
simple_classifier_config.hidden_dim = 32
simple_classifier_config.output_dim = 1
simple_classifier_config.num_layers = 2
simple_classifier_config.dropout = 0.1

In [15]:
mda_classifier = SimpleMDAClassifier(simple_classifier_config)

2024-02-11 17:02:15,293 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-11 17:02:15,295 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout


## Optimizer

In [16]:
classifier_optimizer_config = OptimizerConfig()
classifier_optimizer_config.optimizer = torch.optim.Adam
classifier_optimizer_config.criterion = torch.nn.BCEWithLogitsLoss()
classifier_optimizer_config.lr = 0.01
classifier_optimizer_config.batch_size = 32
classifier_optimizer_config.n_epoch = 50
classifier_optimizer_config.exp_name = "adam optimizer"
classifier_optimizer_config.save = False
classifier_optimizer_config.save_path = None
classifier_optimizer_config.device = device
classifier_optimizer_config.report_size = 10  # batch to report ratio
classifier_optimizer_config.threshold = 0.5

## Train Test Approach

### Train

In [17]:
train_result = SimpleTrainer().train(model=mda_classifier,
                                     data=train_data,
                                     config=classifier_optimizer_config)

2024-02-11 17:02:15,318 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-11 17:02:15,321 [INFO] moving data and model to cpu
2024-02-11 17:02:15,364 [INFO] loss: 0.0192    [1,    10]
2024-02-11 17:02:15,381 [INFO] loss: 0.0161    [1,    20]
2024-02-11 17:02:15,398 [INFO] loss: 0.0142    [2,    10]
2024-02-11 17:02:15,423 [INFO] loss: 0.0128    [2,    20]
2024-02-11 17:02:15,442 [INFO] loss: 0.0136    [3,    10]
2024-02-11 17:02:15,458 [INFO] loss: 0.0097    [3,    20]
2024-02-11 17:02:15,476 [INFO] loss: 0.0105    [4,    10]
2024-02-11 17:02:15,492 [INFO] loss: 0.0112    [4,    20]
2024-02-11 17:02:15,510 [INFO] loss: 0.0093    [5,    10]
2024-02-11 17:02:15,527 [INFO] loss: 0.0104    [5,    20]
2024-02-11 17:02:15,543 [INFO] loss: 0.0099    [6,    10]
2024-02-11 17:02:15,560 [INFO] loss: 0.0085    [6,    20]
2024-02-11 17:02:15,576 [INFO] loss: 0.0083    [7,    10]
2024-02-11 17:02:15,592 [INFO] loss: 0.0094    [7,    20]
2024-02-11 17:02:15,609 [INFO] loss: 0.0080  

### Test

In [18]:
test_result = SimpleTester().test(model=mda_classifier,
                                  data=test_data,
                                  config=classifier_optimizer_config)

2024-02-11 17:02:17,505 [INFO] Running Simple Tester with config : adam optimizer
2024-02-11 17:02:17,511 [INFO] moving data and model to cpu
2024-02-11 17:02:17,557 [INFO] Result on Test Data : {'AUC': 0.9718271184579055, 'ACC': 0.9111111111111111, 'F1 Score': 0.9110671936758893, 'AUPR': 0, 'Loss': 0.2504637518690692}


In [19]:
test_result.get_result()

{'AUC': 0.9718271184579055,
 'ACC': 0.9111111111111111,
 'F1 Score': 0.9110671936758893,
 'AUPR': 0,
 'Loss': 0.2504637518690692}

## Cross Validation

In [20]:
trainer = SimpleTrainer()
tester = SimpleTester()
factory = SimpleMDAClassifierFactory(simple_classifier_config)
spliter = SimplePytorchDataTrainTestSplit(data)
cross_validation(k=5, data_size=data.X.shape[0], train_test_spliter=spliter, model_factory=factory,
                    trainer=trainer, tester=tester, config=classifier_optimizer_config)

2024-02-11 17:02:17,580 [INFO] Initializing SimpleMDAClassifierFactory with model : simple classifier
2024-02-11 17:02:17,582 [INFO] Initializing SimplePytorchDataTrainTestSplit
2024-02-11 17:02:17,584 [INFO] Start 5-fold Cross Validation with config : adam optimizer
2024-02-11 17:02:17,587 [INFO] ---- Fold 1 ----
2024-02-11 17:02:17,590 [INFO] Initializing SimplePytorchData with X shape : torch.Size([719, 64]) and y shape : torch.Size([719, 1])
2024-02-11 17:02:17,593 [INFO] Initializing SimplePytorchData with X shape : torch.Size([179, 64]) and y shape : torch.Size([179, 1])
2024-02-11 17:02:17,595 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-11 17:02:17,596 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout
2024-02-11 17:02:17,598 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-11 17:02:17,600 [INFO] moving data and model to cpu
2024-02-11 17:02:17,